# Data management (individual CSV files $\to$ joint CSV file)

STAGE 2 OF THE DATA PIPELINE

Things that happen in this script:
1. Combine multiple Atlas project CSVs (output by Stage 1) into a single dataframe
2. Double-check to remove any duplicate rows, or rows from certain "test" chats
3. Perform post-hoc codebook edits (merge "errorLine" into "errorLocation")
4. Extract "interaction" (or "conversation," or "issue")-level information (e.g., outcome) and store it
5. Store "blinded" versions of the codes (where information such as "conversation outcome" is blinded; this is useful for some analyses later on)
6. Construct a dataframe indicating which annotators annotated which documents (FIXME programmatically check this against the one in Google Drive)

Clean up quote endpoints and extract metadata from multiple Atlas projects, and combine them into a single file

## Flags

In [ ]:
output = True

input_versions = [14, 15, 16]
output_version = 7

## Baseline setup

In [ ]:
import os
import numpy as np
import pandas as pd
from itertools import chain, combinations

In [ ]:
input_files = ["../output/v{}/human-readable-annotations.csv".format(input_version) for input_version in input_versions]

output_parent_dir = "../output/clean"
output_child_dir = "v{}".format(output_version)
output_dir = os.path.join(output_parent_dir, output_child_dir)

output_file = "annotations_data.csv"
sample_output_file = "small_" + output_file
output_coders = "coders_per_document.csv"

# Long-form dataframe of annotations from document-annotator pairs without well-defined
# conversation sentinels (excluded from all subsequent dataframes because I can't infer
# conversation IDs, types, and outcomes for any of them)
excluded_rows_file = "excluded_annotations.csv"

# List of the chats that were excluded (in the file above)
anomalous_chats_file = "anomalous_chats.csv"

In [ ]:
if output:
    try:
        os.mkdir(output_parent_dir)
    except FileExistsError:
        print("High-level output directory already exists; no action taken.")
    
    try:
        os.mkdir(output_dir)
    except FileExistsError:
        print("WARNING: low-level output directory already exists. You might want to increment your version number.")

In [ ]:
# util for displaying dataframes
# the defaults are actually 60 & 20, but that gets annoying
def show(da, rows = 20, cols = 20):
    pd.set_option("display.max_rows", rows)
    pd.set_option("display.max_columns", cols)
    display(da)
    pd.reset_option("max_rows")
    pd.reset_option("max_columns")

## Read in the data

In [ ]:
das = [pd.read_csv(input_file, index_col = 0) for input_file in input_files]

Fix the columns and datatypes

In [ ]:
dtypes = {'quote.text' : 'string', 
          'annotation.code' : 'string', 
          'annotation.creatingUser' : 'string',
          'annotation.creationDateTime' : 'string', 
          'quote.startPosition' : np.int64, 
          'quote.endPosition' : np.int64, 
          'quote.creatingUser' : 'string', 
          'quote.creationDateTime' : 'string', 
          'quote.modifyingUser' : 'string', 
          'quote.modifiedDateTime' : 'string', 
          'document.name' : 'string', 
          'document.creatingUser' : 'string', 
          'document.creationDateTime' : 'string', 
          'document.modifyingUser' : 'string', 
          'document.modifiedDateTime' : 'string',
          'document.plainTextPath' : 'string', 
          'document.richTextPath' : 'string', 
          'annotation.guid' : 'string', 
          'annotation.codeRef.guid' : 'string', 
          'quote.guid' : 'string', 
          'document.guid' : 'string', 
          'quote.paragraphStartPosition' : np.int64, 
          'quote.paragraphEndPosition' : np.int64,
          'quote.paragraphText' : 'string', 
          'quote.speaker' : np.int64, 
          'quote.speakerIsLearner' : bool}
defaults = {np.int64 : "-1", "string" : "N/A"}

In [ ]:
for da in das:
    for col in dtypes.keys():
        if not col in da.columns:
            da[col] = defaults[dtypes[col]] # to deal with missing info from Version 4

Now combine things

In [ ]:
da = pd.concat(das).drop_duplicates().reset_index(drop=True)
#da = pd.read_csv(input_files[0], index_col = 0)

In [ ]:
show(da, rows=4)

## Remove test rows
This should now be redundant to steps completed in `data-management_qdpx-to-csv.ipynb`, so just check that the printed output agrees with that.

In [ ]:
# we want to keep these
print(da[da["document.name"].str.endswith(".txt")].shape)

# but not these
print(da[~da["document.name"].str.endswith(".txt")].shape)

In [ ]:
da1 = da[da["document.name"].str.endswith(".txt")]

In [ ]:
da1["annotation.creatingUser"].value_counts()

In [ ]:
print(da1.iloc[10]["quote.endPosition"] - da1.iloc[10]["quote.startPosition"])
print(len(da1.iloc[10]["quote.text"]))

In [ ]:
da1.iloc[10]["quote.text"]

## Post-hoc codebook edits

Merge: 
1. "errorLine" and "errorLocation"
2. "aggression" and "frustration"
3. "phrasedAsQuestion" and "phrasedAsUnsure"

In [ ]:
da1["annotation.original_code"] = da1["annotation.code"]
da1["annotation.code"] = da1["annotation.code"].replace(
    "General message attributes > contentDomain > errorLine", 
    "General message attributes > contentDomain > errorLocation")
da1["annotation.code"] = da1["annotation.code"].replace(
    "Attitude, tone, or mood > expressNegativity > aggression", 
    "Attitude, tone, or mood > expressNegativity > frustration")
da1["annotation.code"] = da1["annotation.code"].replace(
    "Explanations and help > confidenceLevel > phrasedAsQuestion", 
    "Explanations and help > confidenceLevel > phrasedAsUnsure")

In [ ]:
annls = np.sort(da1["annotation.code"].unique())
len(annls)

In [ ]:
annmp = {x : i for i, x in enumerate(annls)}

## Remove any duplicates and sort the dataframe

In [ ]:
# check for duplicates
# even though it would be counter-intuitive later on, we need to sort by annotator here
# in order for the conversation labelling to run smoothly
important_cols = ["document.name", "annotation.creatingUser", 
                  "quote.startPosition", "quote.endPosition", 
                  "annotation.code"]
print(da1.shape)
print(da1[important_cols].drop_duplicates().shape)
print("Duplicates present? {}.".format("No" if len(da1) == len(da1[important_cols].drop_duplicates()) else "Yes"))

In [ ]:
# get rid of duplicates (this also sorts the dataframe yay)
da2 = da1.reset_index().rename(columns={"index" : "da1.idx"}).groupby(by=important_cols).agg("first")

# reindex and revert the column ordering
da2 = da2.reset_index()[da1.columns.insert(0, "da1.idx")]
da2.head(3)

In [ ]:
da2.shape

In [ ]:
(da2.sort_values(by=important_cols) == da2).all().all() # check that it's sorted

## Label the "interactions/conversations" (requests) and their outcomes

### First check for issues.

In [ ]:
# given the indices for request and resolveRequest annotations in a chat, 
# validate that they bookend valid ranges of annotations
def validate_interactions(reqls, resls):
    # edge case
    if len(reqls) == 0 or len(resls) == 0:
        return False
    
    i = 0
    j = 0
    
    # for each interaction
    while i < len(reqls) and j < len(resls):
        # check that the starting point is valid
        if reqls[i] > resls[j]:
            return False
        
        # read all requests in this interaction
        while i < len(reqls) and j < len(resls) and reqls[i] < resls[j]:
            i += 1
        
        # move to the next interaction
        j += 1
        
    return i == len(reqls) and j == len(resls)

In [ ]:
# test it
validate_interactions([10, 18, 44], [31, 110])

In [ ]:
# check for missing data (documents with missing interaction info)
invalid_chats = {}

In [ ]:
for doc in da2["document.name"].unique():
    users = da2[da2["document.name"] == doc]["annotation.creatingUser"].unique()
    for user in users:
        da3 = da2[(da2["document.name"] == doc) & (da2["annotation.creatingUser"] == user)]
        da3 = da3.reset_index().rename(columns={"index" : "da1.idx"})
    
        requestls = da3[da3["annotation.code"].str.startswith(
            "Big picture of an interaction > request")].index.values.tolist()
        
        resolvedls = da3[da3["annotation.code"].str.startswith(
            "Big picture of an interaction > resolveRequest")].index.values.tolist()
        
        if not validate_interactions(requestls, resolvedls):
            print("[Failed validation] Document {} annotated by {}".format(doc, user))
            invalid_chats[(doc, user)] = (requestls.copy(), resolvedls.copy())
        
        #problems = []
        #if not validate_interactions(requestls, resolvedls):
        #    problems.append("Failed validation")
        #    printed_something = True
        
        #annotated = True
        #try:
        #    annotated = document_metadata.loc[doc, user.split(" ")[0]]
        #except(KeyError):
        #    annotated = True # M isn't in the spreadsheet
        
        #if len(problems) > 0:
        #    problems = ", ".join(problems)
        #    print("[{}]{} Document {} annotated by {}".format(problems, " " * (37 - len(problems)), doc, user))
        #    invalid_chats[(doc, user)] = (requestls.copy(), resolvedls.copy(), annotated)

In [ ]:
for (doc, user), (req, res) in invalid_chats.items():
    print( # ("ok" if not ann else "  "), " ",
          doc, "   ", 
          user, " " * (40 - len(doc) - len(user)), 
          req, "vs.", res #, " " * (50 - len(req) - len(res)), 
          )
    #print(req)
    #print(res)
    #print()

In [ ]:
dict_to_da = {(d, u) : {"Request indices" : req, 
                        "Resolved indices" : res, 
                        # "Was annotated" : ann
                       } 
              for (d, u), (req, res) in invalid_chats.items()}
invalid_chats_da = pd.DataFrame.from_dict(data = dict_to_da, orient = "index", dtype=str)
invalid_chats_da = invalid_chats_da.reset_index()
invalid_chats_da = invalid_chats_da.rename(columns={"level_0" : "Document", 
                                                    "level_1" : "Annotator"})
invalid_chats_da

In [ ]:
missing_start = invalid_chats_da["Request indices"] == "[]"
invalid_chats_da["Problem"] = np.where(missing_start, "No start labels", "")

missing_end = invalid_chats_da["Resolved indices"] == "[]"
invalid_chats_da["Problem"] = np.where(missing_end, "No end labels", invalid_chats_da["Problem"])

missing_both = (invalid_chats_da["Request indices"] == "[]") & (invalid_chats_da["Resolved indices"] == "[]")
invalid_chats_da["Problem"] = np.where(missing_both, "No start or end labels", invalid_chats_da["Problem"])

Now do the one-offs, if needed (it wasn't needed).

In [ ]:
invalid_chats_da

Code for testing the chats extracted above

In [ ]:
doc, user = "451n3l6h9l.txt", "A"
invalid_chats[(doc, user)]

In [ ]:
tmp = da2[(da2["document.name"] == doc) & (da2["annotation.creatingUser"] == user)]
tmp = tmp.reset_index().rename(columns={"index" : "da1.idx"})
tmp.shape

In [ ]:
pd.set_option("display.max_colwidth", None)
ii = np.concatenate(invalid_chats[(doc, user)])
ii.sort()
tmp.iloc[ii][["da1.idx", 
              "quote.text", 
              "annotation.code", 
              "annotation.creatingUser", 
              "annotation.creationDateTime", 
              "quote.startPosition", 
              "quote.endPosition"]]

In [ ]:
pd.reset_option("display.max_colwidth")

Remove the chats that failed validation

In [ ]:
invalid_rows = pd.Series(data=zip(da2["document.name"], da2["annotation.creatingUser"]), 
                         index=da2.index).apply(lambda x : x in invalid_chats.keys())
da3 = da2[~invalid_rows].reset_index().rename(columns={"index" : "da2.idx"})
da3

In [ ]:
if output:
    invalid_chats_da.to_csv(os.path.join(output_dir, anomalous_chats_file))
    da2[invalid_rows].to_csv(os.path.join(output_dir, excluded_rows_file))

### Now actually label them

In [ ]:
# given the indices for request and resolveRequest annotations in a chat, 
# output the number of interactions and their bounds (as indices)
# this should only be run on inputs that pass validate_interactions()
def extract_interactions(reqls, resls):
    interactions = []
    
    i = 0
    j = 0
    
    # for each interaction
    while i < len(reqls) and j < len(resls):
        k = i
        
        # read all requests in this interaction
        while i < len(reqls) and j < len(resls) and reqls[i] < resls[j]:
            i += 1
        
        interactions.append((reqls[k:i], resls[j])) # inclusive, exclusive
        
        # move to the next interaction
        j += 1
        
    return interactions

In [ ]:
# test it
print(extract_interactions([1, 2, 5], [3, 6]))
print(extract_interactions([10, 20, 50], [30, 60]))

In [ ]:
interaction_idx = pd.Series(data=-1, index=da3.index)         # index for interactions in a document
interaction_len = pd.Series(data=0, index=da3.index)          # length of this annotation's interaction
strict_interaction_len = pd.Series(data=0, index=da3.index)   # length of this annotation's interaction
requests = pd.Series(data="N/A", index=da3.index)             # comma-separated list of requests for 
                                                              # this interaction (leaf annotation only)
outcomes = pd.Series(data="N/A", index=da3.index)             # "S" for successes and "F" for failures
well_defined = pd.Series(data=False, index=da3.index)         # True iff the interaction number is unambiguous

assert(len(outcomes) == len(da3))
interaction_idx.value_counts()

The way I've implemented this currently, any Interaction has exactly one combination of Requests. This keeps the code/analysis simpler, but it also means that for interactions where a new request is made partway through, all of the previous annotations in the interaction will still identify with that request type even though it hasn't been made yet.

This might change depending on the requirements of later analyses.

Update: In all recent versions of the input data, each Interaction should have *exactly* one Request, so this shouldn't be a problem.

In [ ]:
assert((da3.sort_values(by=important_cols) == da3).all().all()) # check that it's still sorted

In [ ]:
for doc in da3["document.name"].unique():
    users = da3[da3["document.name"] == doc]["annotation.creatingUser"].unique()
    for user in users:
        da4 = da3[(da3["document.name"] == doc) & (da3["annotation.creatingUser"] == user)]
        da4 = da4.reset_index().rename(columns={"index" : "da3.idx"})
    
        requestls = da4[da4["annotation.code"].str.startswith(
            "Big picture of an interaction > request")].index.values.tolist()
        
        resolvedls = da4[da4["annotation.code"].str.startswith(
            "Big picture of an interaction > resolveRequest")].index.values.tolist()
        
        # write interaction info into the Serieses defined above
        interactionls = extract_interactions(requestls, resolvedls)
        persistent_weak_start = 0 # start index that lumps preceding ambiguous annotations into each interaction
        for n, (ireqls, ires) in enumerate(interactionls): # for each interaction
            # get the request type(s)
            reqls = da4.loc[ireqls, "annotation.code"].str.split(" > ").str[-1].unique() # list of req
            reqls.sort() # alphabetize
            reqstr = ", ".join(reqls)
            
            # get the outcome
            res = da4.loc[ires, "annotation.code"].split(" > ")[-1][0].capitalize() # "S" or "F"
            assert(res in {"S", "F"}) # double-check

            # get the earliest start index and latest end index of the interaction (wrt da4)
            start4 = ireqls[0]
            while (start4 > 0 and 
                   da4.loc[start4-1, "quote.startPosition"] >= da4.loc[ireqls[0], "quote.startPosition"]):
                start4 -= 1

            stop4 = ires # inclusive
            while (stop4+1 < len(da4) and # da4 has consecutive indexing 
                   da4.loc[stop4+1, "quote.endPosition"] <= da4.loc[ires, "quote.endPosition"]):
                stop4 += 1

            # translate da4 indices into da3 indices (the ordering property should be preserved)
            start = da4.loc[start4, "da3.idx"]
            stop = da4.loc[stop4, "da3.idx"] + 1 # exclusive (da3 also has consecutive indexing)
            
            weak_start = da4.loc[persistent_weak_start, "da3.idx"]
            weak_stop = stop if n < len(interactionls) - 1 else da4.loc[len(da4)-1, "da3.idx"] + 1 # exclusive
            
            # write the information to the output Serieses
            interaction_idx.iloc[weak_start:weak_stop] = n
            interaction_len.iloc[weak_start:weak_stop] = weak_stop - weak_start
            requests.iloc[weak_start:weak_stop] = reqstr
            outcomes.iloc[weak_start:weak_stop] = res
            
            well_defined[start:stop] = True
            strict_interaction_len.iloc[start:stop] = stop - start
            
            # update the weak interaction boundary index
            persistent_weak_start = stop4+1

In [ ]:
# this was the old way of checking this; it returned 31725 on input versions [14, 15, 16] and they match, yay!
#well_defined = (interaction_idx != -1) 
well_defined.value_counts()

Not all rows belong to a well-defined interaction (some are in-between sentinels).

We clean up by grouping orphaned rows with the next interaction (not the previous one, since they can't have causally impacted the result) where possible. However, orphaned rows that are actually at the end of the chat document will be grouped with the previous interaction out of necessity.

We also leave an `interaction.strict` column providing context for these rows.

Time permitting, I'll come back here and leave an `interaction.alt_number` column too.

In [ ]:
# da3 = da3[well_defined].reset_index(drop=True)
# interaction_idx = interaction_idx[well_defined].reset_index(drop=True)
# requests = requests[well_defined].reset_index(drop=True)
# outcomes = outcomes[well_defined].reset_index(drop=True)

In [ ]:
interaction_idx.value_counts()

Write the above results into the dataframe.

In [ ]:
da3["interaction.number"] = interaction_idx
da3["interaction.len"] = interaction_len
da3["interaction.requests"] = requests
da3["interaction.outcome"] = outcomes
da3["interaction.strict"] = well_defined

da3["interaction.strict_len"] = strict_interaction_len

In [ ]:
interaction_idx.value_counts()

In [ ]:
outcomes.value_counts()

In [ ]:
requests.value_counts()

## Make columns for blinded code labels

In [ ]:
# codes column, but with outcomes masked (requests are unmasked)
da3["annotation.code.noOutcome"] = np.where(
    da3["annotation.code"].str.startswith("Big picture of an interaction > resolveRequest"), 
    "Big picture of an interaction > resolveRequest", 
    da3["annotation.code"])

# codes column, but with both requests and outcomes masked
da3["annotation.code.noRequestOutcome"] = np.where(
    da3["annotation.code"].str.startswith("Big picture of an interaction > request"), 
    "Big picture of an interaction > request", 
    da3["annotation.code.noOutcome"])

## Output

In [ ]:
da3["document.creationDateTime"].value_counts()

In [ ]:
if output:
    da3.to_csv(os.path.join(output_dir, output_file))
    da3.head(20).to_csv(os.path.join(output_dir, sample_output_file)) # for easy visualization on GitHub

## Figure out who coded which documents

FIXME check these against the spreadsheet

In [ ]:
indicators = np.array([[coder, (lambda x, coder=coder : (x == coder).any())] # new syntax yay
                       for coder in da3["annotation.creatingUser"].unique()])

for i in range(indicators.shape[0]): # loop over the functions
    print("(function) {}:".format(indicators[i, 0])) # name associated with the function we're testing
    for j in range(indicators.shape[0]): # loop over the inputs
        print("== (input) {}? \t\t\t{}".format(indicators[j, 0], indicators[i, 1](pd.Series([indicators[j, 0]])))) # test input j against function i

In [ ]:
# sampling
n = 3000
step = 10
minida1 = da3.iloc[::step, :].head(n)

In [ ]:
#display(da1.head(1000).groupby(by="document.name", as_index=False).agg({"annotation.creatingUser" : lambda x : (x == "A").any()}))

n = 300
minida2 = minida1.head(n)

for k in range(indicators.shape[0]):
    print("{}:".format(indicators[k, 0]))
    show(minida2.groupby(by="document.name", as_index=False).agg({"annotation.creatingUser" : indicators[k, 1]}))

for dname in minida2["document.name"].unique():
    tmpda = minida2[minida2["document.name"] == dname]
    print("Document: {}".format(dname))
    print(tmpda["annotation.creatingUser"].unique())

In [ ]:
documentinfo = list(filter(lambda col : col.startswith("document."), da3.columns))
dda = da3.groupby(by="document.name", 
                  as_index=False).agg({**{col : [lambda x : x.iloc[0]] for col in documentinfo}, 
                                       **{"annotation.creatingUser" : indicators[:, 1]}})
dda.columns = np.concatenate((dda.columns.get_level_values(level=0)[:-len(indicators[:, 0])], # other metadata fields
                              indicators[:, 0])) # people names
dda.head(3) # don't worry, not all of them are True

In [ ]:
# this prints the number of documents coded by each combination of people (have fun with inclusion-exclusion!)
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s)+1))

coders = indicators[:, 0]
for subset in powerset(coders):
    print(", ".join(list(subset)), " : ", dda[dda[list(subset)].all(axis=1)].shape[0])

In [ ]:
# documents seen by all 4 coders
subset = list(powerset(coders))[-1]
print("\n".join(list(dda[dda[list(subset)].all(axis=1)]["document.name"]))) # note this doesn't re-index

In [ ]:
dda[dda[list(subset)].all(axis=1)]

In [ ]:
# this prints how many annotations each person made on each document
for dname in da3["document.name"].unique():
    tmpda = da3[da3["document.name"] == dname]
    print("Document: {}".format(dname))
    print(tmpda["annotation.creatingUser"].value_counts())
    print()

### Output

In [ ]:
if output:
    dda.to_csv(os.path.join(output_dir, output_coders))